##### 1. Fetching data from the dataset


In [9]:
import requests
from bs4 import BeautifulSoup
import json

url = "https://www.gutenberg.org/cache/epub/59824/pg59824-images.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Titles are displayed in <h4> and <h5>
# tags , while the poem themselves are
# displayed in <p> tags.    


# The first few tags are not useful

titles = soup.find_all(['h4', 'h5'])[7:]                                    
poems = soup.find_all('p')[8:]

poems_data = []
for title, poem in zip(titles, poems):
    poems_data.append({
        "title": title.text.strip(),
        "content": poem.text.strip()
    })

with open('Robert_Frost_Poem_Collection.json', 'w') as f:
    json.dump(poems_data, f, indent=4)

# !!! The data is still not proper.
# Additional Manual cleaning of data needs to be done.

# The file name is intentionally written as Robert_Frost_Poem_Collection and not Robert_Frost_Poem_Collections 
# so as not to accidentally write over the previous file.


#### 2. Tokenization

In [10]:
# This is the sample implementation of BPE in tiktoken (https://github.com/openai/tiktoken/blob/main/tiktoken/_educational.py)
# It is modified to work with our code.


"""This is an educational implementation of the byte pair encoding algorithm."""
import collections
import regex

gpt2_regex = (r"""'s|'t|'re|'ve|'m|'ll|'d| ?[\p{L}]+| ?[\p{N}]+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""" )
gpt4_regex = (r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""")

class SimpleBytePairEncoding:
    def __init__(self, *, mergeable_ranks: dict[bytes, int]) -> None:
        """Creates an Encoding object."""
        # A regex pattern string that is used to split the input text
        self.pat_str = gpt4_regex
        # A dictionary mapping token bytes to their ranks. The ranks correspond to merge priority
        self.mergeable_ranks = mergeable_ranks

        self._decoder = {token: token_bytes for token_bytes, token in mergeable_ranks.items()}
        self._pat = regex.compile(gpt4_regex)

    def encode(self, text: str) -> list[int]:
        """Encodes a string into tokens.

        >>> enc.encode("hello world")
        [388, 372]
        """
        # Use the regex to split the text into (approximately) words
        words = self._pat.findall(text)
        tokens = []
        for word in words:
            # Turn each word into tokens, using the byte pair encoding algorithm
            word_bytes = word.encode("utf-8")
            word_tokens = bpe_encode(self.mergeable_ranks, word_bytes)
            tokens.extend(word_tokens)
        return tokens
    
    def decode_bytes(self, tokens: list[int]) -> bytes:
        """Decodes a list of tokens into bytes.

        >>> enc.decode_bytes([388, 372])
        b'hello world'
        """
        return b"".join(self._decoder[token] for token in tokens)

    def decode(self, tokens: list[int]) -> str:
        """Decodes a list of tokens into a string.

        Decoded bytes are not guaranteed to be valid UTF-8. In that case, we replace
        the invalid bytes with the replacement character "�".

        >>> enc.decode([388, 372])
        'hello world'
        """
        return self.decode_bytes(tokens).decode("utf-8", errors="replace")

    @staticmethod
    def train(training_data: str, vocab_size: int):
        """Train a BPE tokeniser on some data!"""
        mergeable_ranks = bpe_train(data=training_data, vocab_size=vocab_size)
        return SimpleBytePairEncoding(mergeable_ranks=mergeable_ranks)


def bpe_encode(mergeable_ranks: dict[bytes, int], input: bytes) -> list[int]:
    parts = [bytes([b]) for b in input]
    while True:

        # Iterate over all pairs and find the pair we want to merge the most
        min_idx = None
        min_rank = None
        for i, pair in enumerate(zip(parts[:-1], parts[1:])):
            rank = mergeable_ranks.get(pair[0] + pair[1])
            if rank is not None and (min_rank is None or rank < min_rank):
                min_idx = i
                min_rank = rank

        # If there were no pairs we could merge, we're done!
        if min_rank is None:
            break
        assert min_idx is not None

        # Otherwise, merge that pair and leave the rest unchanged. Then repeat.
        parts = parts[:min_idx] + [parts[min_idx] + parts[min_idx + 1]] + parts[min_idx + 2 :]

    tokens = [mergeable_ranks[part] for part in parts]
    return tokens


def bpe_train(data: str, vocab_size: int) -> dict[bytes, int]:
    # First, add tokens for each individual byte value
    if vocab_size < 2**8:
        raise ValueError("vocab_size must be at least 256, so we can encode all bytes")
    ranks = {}
    for i in range(2**8):
        ranks[bytes([i])] = i

    # Splinter up our data into lists of bytes
    # data = "Hello world"
    # words = [
    #     [b'H', b'e', b'l', b'l', b'o'],
    #     [b' ', b'w', b'o', b'r', b'l', b'd']
    # ]
    words: list[list[bytes]] = [
        [bytes([b]) for b in word.encode("utf-8")] for word in regex.findall(gpt4_regex, data)
    ]

    # Now, use our data to figure out which merges we should make
    while len(ranks) < vocab_size:
        # Find the most common pair. This will become our next token
        stats = collections.Counter()
        for piece in words:
            for pair in zip(piece[:-1], piece[1:]):
                stats[pair] += 1

        most_common_pair = max(stats, key=lambda x: stats[x])
        token_bytes = most_common_pair[0] + most_common_pair[1]
        token = len(ranks)
        # Add the new token!
        ranks[token_bytes] = token

        # Now merge that most common pair in all the words. That is, update our training data
        # to reflect our decision to make that pair into a new token.
        new_words = []
        for word in words:
            new_word = []
            i = 0
            while i < len(word) - 1:
                if (word[i], word[i + 1]) == most_common_pair:
                    # We found our pair! Merge it
                    new_word.append(token_bytes)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            if i == len(word) - 1:
                new_word.append(word[i])
            new_words.append(new_word)
        words = new_words

    return ranks

In [11]:
import json

dataset = json.load(open('Robert_Frost_Poem_Collections.json'))

titles = []                             #titles of the poems
contents = []                           #actual content of the poems
for item in dataset:
  titles.append(item.get('title'))
  contents.append(item.get('content'))

training_text = ''.join(contents)

#Tokenizer object , which will encode and decode
#our text to tokens.

tokenizer = SimpleBytePairEncoding(mergeable_ranks=dict()).train(training_text,vocab_size=1000)                           

In [15]:
message = "How poetic of a road this is ? (I must say ; for I am amused)"
encoded_message = tokenizer.encode(message)
decoded_message = tokenizer.decode(encoded_message)
print(f"Encoded message  : {encoded_message} ")
print(f"Original message : {message} ")
print(f"Decoded message  : {decoded_message} ")

print('\n')

print(f"Original size     : {len(message)} ")
print(f"Encoded size      : {len(encoded_message)} ")
print(f"Compression Ratio : {len(decoded_message)/len(encoded_message):.4f}")

Encoded message  : [72, 285, 307, 111, 313, 736, 299, 259, 930, 537, 361, 32, 63, 32, 40, 73, 643, 396, 32, 59, 346, 286, 867, 867, 784, 281, 41] 
Original message : How poetic of a road this is ? (I must say ; for I am amused) 
Decoded message  : How poetic of a road this is ? (I must say ; for I am amused) 


Original size     : 61 
Encoded size      : 27 
Compression Ratio : 2.2593
